In [1]:
import tensorflow as tf

# Papar's model

In [1]:
def inference(images0):
    """Build the CNN model.
    Args: images0: Images placeholder, from inputs(). 
    Returns: sigmoid_linear: Output tensor with the computed probabilities. """
    images = tf.reshape(images0,[-1,267,9,1])
    
    # conv1 
    with tf.variable_scope('conv1') as scope:
        weights = tf.Variable(tf.truncated_normal([16,2,1,10], stddev=0.1))
        conv = tf.nn.conv2d(images, weights, [1,1,1,1], padding='VALID')
        biases = tf.Variable(tf.constant(0.1, shape=[10]))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name=scope.name)
        
    # pool1 
    pool1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME', name='pool1')
    
    # conv2
    with tf.variable_scope('conv2') as scope:
        weights = tf.Variable(tf.truncated_normal([11,3,10,20], stddev=0.1))
        conv = tf.nn.conv2d(pool1, weights, [1,1,1,1], padding='VALID')
        biases = tf.Variable(tf.constant(0.1, shape=[20]))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(pre_activation, name=scope.name)
        
    # pool2 
    pool2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME', name='pool2')
    
    # fully conneted1
    with tf.Variable_scope('fully-connected1') as scope:
        reshape = tf.reshape(pool2, [-1,58*1*20])
        weights = tf.Variable(tf.truncated_normal([58*1*20, 256], stddev=0.1))
        biases = tf.Variable(tf.constant(0.1, shape=[256]))
        local = tf.nn.relu(tf.matmul(reshape, weights) + biases, name=scope.name)
        
    # fully connected2
    with tf.variable_scope('fully-connected2') as scope:
        # dropout
        local3_drop = tf.nn.dropout(local, 0.5)
        weights = tf. Variable(tf.truncated_normal([256, num_classes], stddev=0.1))
        biases = tf.Variable(tf.constant(0.1, shape=[num_classes]))
        sigmoid_linear = tf.nn.sigmoid(tf.matmul(local3_drop, weights) + biases, name=scope.name)
        
    return sigmoid_linear

def loss(logits, labels):
    """Calculates the loss from the logits and the labels. 
    Args: 
    logits: Logits from inference(), float - [batch_size, num_classes].
    labels: Labels tensor, int32 - [batch_size, num_classes].
    Returns: cross_entropy: Loss tensor of type float. """
    cross_entropy = -tf.reduce_sum(labels*tf.log(logits+1e-10)+(1-labels)*tf.log(1-logits+1e-10))
    return cross_entropy

def evaluation(logits, labels, threshold):
    """Evaluate the quality of the logits at predicting the label.
    Args: logits: Logits from inference(), float - [batch_size, num_classes]. 
    labels: Labels tensor, int32 - [batch_size, num_classes]. 
    threshold: Threshold applied to the logits. 
    Returns: accuracy: Compute precision of predicting. """
    pred = tf.cast(tf.greater(logits, threshold), "float")
    correct_predition = tf.cast(tf.equal(pred, labels), "float")
    accuracy = tf.reduce_mean(correct_predition)
    return accuracy

def training(loss, learning_rate):
    
    """Sets up the training Ops. 
    Creates an optimizer and applies the gradients to all trainable variables. 
    Args: loss: Loss tensor, from loss(). 
    learning_rate: The learning rate to use for gradient descent. 
    Returns: train_op: The Op for training. """
    # Gradient descent optimizer with given learning rate
    optimizer = tf.train.AdamOptimizer(learning_rate)
    
    # Use the optimizer to apply the gradients that minimize the loss
    train_op = optimizer.minimize(loss)
    return train_op

# Our model

In [ ]:
num_classes = 0

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(10, (16, 2), activation='relu', padding='valid', input_shape=(267, 9, 1), name="conv1"),
    tf.keras.layers.MaxPooling2D((2, 2), name="pool1"),
    tf.keras.layers.Conv2D(20, (11, 3), activation='relu', padding='valid', name="conv2"),
    tf.keras.layers.MaxPooling2D((2, 2), name="pool2"),
    tf.keras.layers.Flatten(name="flatten"),
    tf.keras.layers.Dense(256, activation='relu', name="fc1"),
    tf.keras.layers.Dropout(0.5, name="dropout"),
    tf.keras.layers.Dense(num_classes, activation='sigmoid', name="fc2")
])

model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ (None, 252, 8, 10)     │           330 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1 (MaxPooling2D)            │ (None, 126, 4, 10)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 116, 2, 20)     │         6,620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool2 (MaxPooling2D)            │ (None, 58, 1, 20)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1160)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 256)            │       297,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 0)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 304,166 (1.16 MB)

 Trainable params: 304,166 (1.16 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizer.Adam(learning_rate=0.01),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
import numpy as np
# Example: Random data (replace with actual dataset)
X_train = np.random.rand(10000, 267, 9, 1)  # 100 training samples
y_train = np.random.randint(0, 1, size=(10000, 2))  # Random binary labels

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=16)


Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.3211 - loss: 0.6942
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.3002 - loss: 0.6931
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.4690 - loss: 0.6933
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.2672 - loss: 0.6930
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.4816 - loss: 0.6933
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.2656 - loss: 0.6931
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.3646 - loss: 0.6931
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.3062 - loss: 0.6933
Epoch 9/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.3742 - loss: 0.6932
Epoch 10/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.3499 - loss: 0.6933


In [ ]:
X_test = np.random.rand(1000, 267, 9, 1)  # 20 test samples
y_test = np.random.randint(0, 1, size=(1000, 2))

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2415 - loss: 0.6931
Test Loss: 0.6931, Test Accuracy: 0.2590
